# 🌐 Riemannian Optimization — Optimization on Manifolds

> “Optimization constrained to a curved space — not by projections, but by geometry.”

---

### 🎯 Objectives
In this notebook we explore **Riemannian Gradient Descent (RGD)** — a generalization of gradient descent to *manifolds* (curved spaces).  
We will:

- Build intuition for optimization *on curved spaces* (e.g., spheres, Stiefel manifolds).  
- Derive the **Riemannian gradient** and **retraction** mechanism.  
- Implement RGD on:
  1. The **unit sphere** (constraint: ‖x‖ = 1)
  2. The **Stiefel manifold** (orthogonal matrix constraints)
- Visualize **Euclidean vs Riemannian** steps on 3D surfaces.  
- Interactively tune learning rate and visualize convergence along geodesics.

---

### 🧩 References
- Absil, Mahony, Sepulchre (2008): *Optimization Algorithms on Matrix Manifolds*  
- Bécigneul & Ganea (2019): *Riemannian Adaptive Optimization Methods* (ICLR)  
- Amari (1998): *Natural Gradient Works Efficiently in Learning*  
- Edelman et al. (1998): *The Geometry of Algorithms with Orthogonality Constraints*

---

## 💡 What is Riemannian Optimization?

In standard (Euclidean) gradient descent, we minimize $ f(x) $ in $ \mathbb{R}^n $:
$$
x_{t+1} = x_t - \eta \nabla f(x_t)
$$

But what if $ x $ must **stay on a manifold**, e.g. the sphere $ S^{n-1} = \{ x : \|x\| = 1 \} $?

Then:
1. The **tangent space** $ T_x\mathcal{M} $ replaces the ambient space.  
2. The **Riemannian gradient** is the projection of the Euclidean gradient onto $ T_x\mathcal{M} $.  
3. The update uses a **retraction** (or exponential map) to move along the manifold.

$$
x_{t+1} = \text{Retr}_x(-\eta \, \textit{grad}_x f)
$$

- The **Riemannian gradient**: $ \text{grad}_x f = P_{T_x}(\nabla f) $
- The **retraction**: maps tangent vector back to the manifold  
  - On the sphere: $ \text{Retr}_x(v) = \frac{x + v}{\|x + v\|} $

---

### 🧮 Example: Sphere constraint
$$
\min_{\|x\|=1} f(x) = x^\top A x
$$
- Euclidean gradient: $ \nabla f = 2 A x $
- Riemannian gradient: $ \text{grad}_x f = \nabla f - (x^\top \nabla f) x $ 

  (the component of ∇f tangent to the sphere)

### Step-1: Imports and Environment Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from IPython.display import clear_output, display, Markdown
try:
    import ipywidgets as widgets
    from ipywidgets import interact, FloatSlider, IntSlider
except ImportError:
    !pip install ipywidgets
    import ipywidgets as widgets
    from ipywidgets import interact, FloatSlider, IntSlider

### Step-2: Riemannian Gradient Descent on Sphere

In [3]:
def riemannian_gd_sphere(A, x0, lr=0.1, n_iter=50):
    x = x0 / np.linalg.norm(x0)
    path = [x]
    losses = []
    for t in range(n_iter):
        g_euc = sphere_grad(x, A)
        g_riem = proj_tangent_sphere(x, g_euc)
        x = retr_sphere(x, -lr * g_riem)
        path.append(x)
        losses.append(sphere_cost(x, A))
    return np.array(path), np.array(losses)

### Step-3: Helper Function for Plotting Utility

In [4]:
def proj_tangent_sphere(x, g):
    """Project Euclidean gradient g onto tangent space of unit sphere at x."""
    return g - np.dot(x, g) * x

def retr_sphere(x, v):
    """Retract tangent vector v back onto sphere."""
    y = x + v
    return y / np.linalg.norm(y)

def sphere_cost(x, A):
    """Quadratic cost on sphere: f(x) = x^T A x"""
    return x.T @ A @ x

def sphere_grad(x, A):
    """Euclidean gradient."""
    return 2 * A @ x

def plot_sphere(A, path=None, title="Riemannian GD on Sphere"):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')
    u = np.linspace(0, 2*np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    xs = np.outer(np.cos(u), np.sin(v))
    ys = np.outer(np.sin(u), np.sin(v))
    zs = np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(xs, ys, zs, color='lightgray', alpha=0.3, linewidth=0)
    if path is not None:
        ax.plot(path[:,0], path[:,1], path[:,2], 'r-', lw=2, label='trajectory')
        ax.scatter(path[0,0], path[0,1], path[0,2], c='blue', s=60, label='start')
        ax.scatter(path[-1,0], path[-1,1], path[-1,2], c='black', marker='*', s=100, label='end')
    ax.set_title(title)
    ax.set_box_aspect([1,1,1])
    ax.legend()
    plt.show()


### Step-4: Interactive Riemannian GD on Sphere

In [5]:
def interactive_sphere_rgd(lr=0.2, n_iter=50):
    clear_output(wait=True)
    A = np.diag([1.0, 0.5, -0.2]) + 0.1*np.random.randn(3,3)
    A = (A + A.T)/2
    x0 = np.random.randn(3)
    path, losses = riemannian_gd_sphere(A, x0, lr=lr, n_iter=n_iter)
    display(Markdown(f"### Riemannian Gradient Descent on Sphere\nLearning rate: {lr}, Iterations: {n_iter}"))
    plot_sphere(A, path)
    plt.figure(figsize=(6,4))
    plt.plot(losses)
    plt.xlabel("Iteration"); plt.ylabel("Loss"); plt.title("Loss vs Iteration"); plt.grid(True, ls='--', lw=0.4)
    plt.show()

interact(interactive_sphere_rgd,
         lr=FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='lr'),
         n_iter=IntSlider(value=50, min=10, max=200, step=5, description='n_iter'))


interactive(children=(FloatSlider(value=0.2, description='lr', max=1.0, min=0.01, step=0.01), IntSlider(value=…

<function __main__.interactive_sphere_rgd(lr=0.2, n_iter=50)>

## 🧮 The Stiefel Manifold $ \text{St}(p, n) $

$$
\text{St}(p, n) = \{ X \in \mathbb{R}^{n \times p} \ | \ X^\top X = I_p \}
$$

Each point $ X $ has orthonormal columns — e.g., a subspace basis.

### Tangent Space
At a point $ X $, the tangent space is:
$$
T_X = \{ Z : X^\top Z + Z^\top X = 0 \}
$$

### Riemannian Gradient
The projection of the Euclidean gradient $ G $ onto $ T_X $ is:
$$
\text{grad}_X f = G - X \cdot \text{sym}(X^\top G)
$$
where $ \text{sym}(M) = \tfrac{1}{2}(M + M^\top) $.

### Retraction (QR-based)
To move along tangent directions while keeping orthonormality:
$$
\text{Retr}_X(V) = \text{qf}(X + V)
$$
where `qf` denotes the Q factor of the QR decomposition.

### Step-5: Stiefel Helpers

In [6]:
def proj_tangent_stiefel(X, G):
    """Project Euclidean gradient G onto tangent space of Stiefel manifold."""
    return G - X @ ((X.T @ G + G.T @ X)/2)

def retr_stiefel(X, V):
    """Retraction via QR decomposition."""
    Y, _ = np.linalg.qr(X + V)
    return Y

def stiefel_cost(X, A):
    return np.trace(X.T @ A @ X)

def stiefel_grad(X, A):
    return 2 * A @ X

### Step-6: RGD on Stiefel Manifold

In [7]:
def riemannian_gd_stiefel(A, X0, lr=0.1, n_iter=50):
    X = X0.copy()
    path = [X]
    losses = []
    for t in range(n_iter):
        G_euc = stiefel_grad(X, A)
        G_riem = proj_tangent_stiefel(X, G_euc)
        X = retr_stiefel(X, -lr * G_riem)
        losses.append(stiefel_cost(X, A))
        path.append(X)
    return path, np.array(losses)

### Step-7: Interactive Stiefel Optimization

In [8]:
def interactive_stiefel_rgd(lr=0.1, n_iter=50, n=3, p=2):
    clear_output(wait=True)
    A = np.random.randn(n,n)
    A = (A + A.T)/2
    X0, _ = np.linalg.qr(np.random.randn(n,p))
    path, losses = riemannian_gd_stiefel(A, X0, lr=lr, n_iter=n_iter)
    plt.figure(figsize=(6,4))
    plt.plot(losses)
    plt.title(f"Riemannian GD on St({p},{n}) - Loss vs Iteration")
    plt.xlabel("Iteration"); plt.ylabel("Loss"); plt.grid(True)
    plt.show()
    display(Markdown(f"Final orthogonality check: XᵀX ≈ I — deviation: {np.linalg.norm(path[-1].T@path[-1]-np.eye(p)):.2e}"))

interact(interactive_stiefel_rgd,
         lr=FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='lr'),
         n_iter=IntSlider(value=50, min=10, max=200, step=5, description='n_iter'),
         n=IntSlider(value=3, min=2, max=5, step=1, description='ambient n'),
         p=IntSlider(value=2, min=1, max=3, step=1, description='cols p'))

interactive(children=(FloatSlider(value=0.1, description='lr', max=0.5, min=0.01, step=0.01), IntSlider(value=…

<function __main__.interactive_stiefel_rgd(lr=0.1, n_iter=50, n=3, p=2)>

## 🔍 Key Takeaways

| Concept | Euclidean | Riemannian |
|----------|------------|------------|
| Space | $ \mathbb{R}^n $ | Manifold $ \mathcal{M} $ |
| Gradient | $ \nabla f $ | Projection onto tangent space $ \text{grad}_x f $ |
| Update | $ x - \eta \nabla f $ | $ \text{Retr}_x(-\eta \, \text{grad}_x f) $ |
| Movement | Straight line | Geodesic curve |
| Constraint enforcement | Projection after update | Intrinsic movement (always on manifold) |

---

### 🌐 Relation to Natural Gradient
The **natural gradient** is a special case of Riemannian gradient descent where the manifold is the *statistical manifold* with metric given by the Fisher information matrix.  
Thus, Riemannian optimization generalizes natural gradient descent beyond probability models.  

$$
\text{NGD: } \tilde{\nabla}_\theta = F^{-1} \nabla_\theta f
\quad \text{is just Riemannian GD on the Fisher manifold.}
$$

## 📚 References

- Absil, P.-A., Mahony, R., Sepulchre, R. (2008). *Optimization Algorithms on Matrix Manifolds.* Princeton University Press.  
- Edelman, A., Arias, T.A., & Smith, S.T. (1998). *The Geometry of Algorithms with Orthogonality Constraints.* SIAM J. Matrix Anal. Appl.  
- Bécigneul, G., & Ganea, O.-E. (2019). *Riemannian Adaptive Optimization Methods.* ICLR.  
- Amari, S. (1998). *Natural Gradient Works Efficiently in Learning.* Neural Computation.